# Basic Flopy

In this notebook, we will blast through basic flopy usage.  There is a lot to cover in flopy, so this will just hit the highlights

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import flopy
import pyemu

Let's jump right in and load an existing model (one we will use for the rest of the week). We will use the `model_ws` (model workspace) to access these files in a different directory:

In [ ]:
model_ws = os.path.join("..","base_model_files")
os.listdir(model_ws)


Do these file types look familiar???

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=model_ws,verbose=True)

In [ ]:
m

individual packages are accessible through their 3-letter extension (except for the basic package, which is `bas6` for some reason :facepalm: )

In [ ]:
m.dis

In each package, the array-type data are housed in special array-handling classes:

In [ ]:
m.dis.top

In [ ]:
m.rch.rech

In [ ]:
m.bas6.strt

List-type data also have special handlers - the list type data is always stored in the `stress_period_data` attribute:

In [ ]:
m.ghb.stress_period_data

Each of these has an `array` attribute to access the actual data values

In [ ]:
arr = m.ghb.stress_period_data.array["bhead"]
cb = plt.imshow(arr[0,0,:,:])
plt.colorbar(cb)

In [ ]:
m.rch.rech.array.shape


And of course, `SFR` has to be difficult

In [ ]:
pd.DataFrame.from_records(m.sfr.reach_data).head()

In [ ]:
pd.DataFrame.from_records(m.sfr.segment_data[0]).head()

## Exporting to shapefiles

You can export the entire model, individual packages or individual attributes:

In [ ]:
m.export("model.shp")
m.dis.export("dis.shp")
m.dis.top.export("top.shp")
m.ghb.stress_period_data.export("ghb.shp")

How does flopy know where the model is in space? The spatial reference:

In [ ]:
m.modelgrid

# changing `model_ws` and writing

It is probably good form to not overwrite the existing files.  To avoid this danger, we can change the model workspace and then write a new set of model files in this location:

In [ ]:
m.change_model_ws("flopy_temp",reset_external=True)
m.write_input()

In [ ]:
os.listdir(m.model_ws)

# Run the model

A little trickery is needed here: we need to get the right MODFLOW binary into this new `model_ws`.  What should we do? 

### DIY: use `shutil` and `os` to copy the `mfnwt` binary into the new `model_ws` (this is a super-common task)

In [ ]:
print(os.listdir("bin")) #hint the platform specific binaries in the bin dir
# your code here:

I prefer to use the pyemu run function here because it is more tolerant of cross platform issues...

In [ ]:
pyemu.os_utils.run("mfnwt freyberg.nam",cwd=m.model_ws)

If you go back to the terminal, you should see the output from MODFLOW-NWT...

# Post-processing

Flopy has lots of support to help use deal with the terrible MDOFLOW file formats

In [ ]:
mflist = flopy.utils.MfListBudget(os.path.join(m.model_ws,m.name+".list"))

In [ ]:
inc_df,cum_df = mflist.get_dataframes(start_datetime="5-11-1955",diff=True)
inc_df

Where did those dates come from???  You can pass a `start_datetime` string to the `get_dataframes()` method to control those better.  In fact, the flopy model instance has its own `start_datetime` attribute for temporally locating the model inputs...

### DIY: plot the NET increment water budget as a bar chart but change the starting datetime to 5 Nov 1955:

In [ ]:
# hint: checkout the options you can pass get_dataframes()
# your code here

We can also load the binary head save file

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(m.model_ws,m.name+".hds"))
hds

In [ ]:
hds.get_times()

In [ ]:
data = hds.get_data() #by defauly, get_data() returns the last entry
data.shape

In [ ]:
hds.plot(mflay=None,totim=None,colorbar=True) # if these args are None, then you get all layers from the last entry

That looks kinda shitty...but we can do better! if you pass the model instance the the `HeadFile` constructor, the plot routines will use the ibound to mask...

### DIY: re-instantiate the `HeadFile` object but pass it the model instance and plot layer 1 heads from both stress periods:

The `HeadFile` class also support writing to shapefiles:

In [ ]:
hds.to_shapefile("hds.shp")

The same type of support is available for the cell-by-cell budget file:

In [ ]:
cbc = flopy.utils.CellBudgetFile(os.path.join(m.model_ws,m.name+".cbc"))
cbc.list_records()

Lets plot up some cell-by-cell info:

In [ ]:
text = "flow right face"
times = cbc.get_times()
fig,axes = plt.subplots(m.nlay,m.nper,figsize=(10,10))
for kper in range(m.nper):
    data = cbc.get_data(text=text,totim=times[kper],full3D=True)[0]
    data = np.ma.masked_where(m.bas6.ibound.array<1,data)
    vmin,vmax = data.min(),data.max()
    
    for k in range(m.nlay):    
        cb = axes[k,kper].imshow(data[k,:,:],vmin=vmin,vmax=vmax)
        axes[k,kper].set_title("{0}, layer {1}, SP {2}".format(text,k+1,kper+1))
        plt.colorbar(cb,ax=axes[k,kper])
plt.tight_layout()
plt.show()


### DIY: change the `model_ws` again (and get the binary into the new `model_ws`).  Then change everyone's fav hydrogeoloical obsession `hk` in each layer by a factor of 10, run the model and *visually* compare the water budget and heads to the base case we ran through above.

# Creating a new model and adding packages

Now we will go through the super painful process of creating a model from scratch

In [ ]:
m = flopy.modflow.Modflow("newmodel",model_ws="newmodel",
                          version="mfnwt",exe_name="mfnwt",
                          external_path="data")

In [ ]:
ncol,nper = 10000,365
tot_length = 100
delr = tot_length / ncol
steady,perlen = [True],[1.0]
for kper in range(nper-1):
    steady.append(False)
    perlen.append(10.0)


In [ ]:
dis = flopy.modflow.ModflowDis(m,nrow=1,ncol=ncol,nlay=1,
                               nper=nper,delr=delr,top=10,
                               botm=0,steady=steady,perlen=perlen)

In [ ]:
m.dis.delr.array[0]

In [ ]:
ibound = np.ones((m.nlay,m.nrow,m.ncol))
ibound[:,:,[0,-1]] = -1 #set constant heads in the first and last column
strt = np.ones_like(ibound)
strt[:,:,-1] = 10 # set the initial heads in the last (constant head) column
bas = flopy.modflow.ModflowBas(m,strt=strt,ibound=ibound)

In [ ]:
upw = flopy.modflow.ModflowUpw(m,hk=10,ss=0.001,sy=0.1)

In [ ]:
nwt = flopy.modflow.ModflowNwt(m)

In [ ]:
oc = flopy.modflow.ModflowOc(m)

To add a boundary condition, we need to generate a dict of stress period key and list-type data values for the wel package, the list type data need to have k, i, j, and flux.  Each BC package has a `get_default_dtype()` method to show you what it is expecting:

In [ ]:
flopy.modflow.ModflowWel.get_default_dtype()

So let's set a well in (about) the center of the model with some pumping in each stress period:

In [ ]:
spd = {}
np.random.seed(111)
for kper in range(m.nper):
    spd[kper] = [0,0,int(m.ncol/2),np.random.uniform(-10,-50,1)]
spd

In [ ]:
wel = flopy.modflow.ModflowWel(m,stress_period_data=spd)

In [ ]:
m.write_input()

In [ ]:
shutil.copy2(os.path.join("bin","mac","mfnwt"),os.path.join(m.model_ws,"mfnwt"))

In [ ]:
pyemu.os_utils.run("mfnwt newmodel.nam",cwd=m.model_ws)

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(m.model_ws,m.name+".hds"))
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111,aspect=1000)
hds.plot(axes=[ax],colorbar=True)


In [ ]:
mflist = flopy.utils.MfListBudget(os.path.join(m.model_ws,m.name+".list"))
inc,cum = mflist.get_dataframes(diff=True)
inc.plot(figsize=(10,10))

Just to show you how power this can be, lets go back up and change the number of columns and stress period...

# DIY: create a 3 layer, 11 row, 11 col model with 365 daily stress periods (first stress period steady-state).  Put constant heads in layer 1 on the left and right with a gradient across the model and a single pumping well in the center of layer 3 and drive the pumping well with random values for each day. Run this model and post-process as above. Copy-and-paste is not cheating!